In [ ]:
import os
# os.chdir('') # set notebook's working directory one up to the project root
print(os.getcwd())
# necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# functions from scripts
# from scripts.fetch_openAQ_data import fetch_openaq_data
# from scripts.process_world_bank_data import process_world_bank_data
# from scripts.analyze_data import combine_datasets

In [ ]:
# fetch and process data - this was done once already ^ and doesn't need to be done anymore
combined_data = pd.read_csv('../data/merged_data.csv')
combined_data.columns
combined_data["Indicator Name"]

In [ ]:
combined_data.head()

In [ ]:
# visualize trends
sns.barplot(data=combined_data, x='Year', y='PM10')
plt.title('Air Quality vs Population Over Time')
plt.show()

sns.lineplot(data=combined_data, x='Year', y='PM10')
plt.title('Air Quality vs Population Over Time')
plt.show()

In [ ]:
# filter rows for PM2.5 and PM10 indicators // SPOILER these graphs were useless
pm_data = combined_data[combined_data['Indicator Name'].str.contains('PM2.5|PM10', na=False)]

# Line plot to compare trends
sns.lineplot(data=pm_data, x='Year', y='WB Value', hue='Indicator Name')
plt.title('PM2.5 and PM10 Trends Over Time')
plt.ylabel('Value')
plt.show()

In [ ]:
# calculate PM2.5/PM10 ratio
combined_data['PM2.5_to_PM10'] = combined_data['WB Value'] / combined_data['PM10']

# line plot for ratio over time
sns.lineplot(data=combined_data, x='Year', y='PM2.5_to_PM10', hue='Country Name')
plt.title('PM2.5 to PM10 Ratio Over Time')
plt.ylabel('PM2.5/PM10')
plt.show()
# shows that pm10 generally < pm2.5, explore further later ?

In [10]:
# further: explore relationships between air quality indicators like PM10 or pm2.5 and socioeconomic factors (e.g., GDP, population)

In [ ]:
from sklearn.preprocessing import StandardScaler

# create a dictionary to map indicators to categories
indicator_categories = {
    'TRADE': ['exports', 'imports', 'trade', 'merchandise'],
    'ECONOMIC': ['gdp', 'gni', 'income', 'economic'],
    'POPULATION': ['population', 'urban', 'rural'],
    'FOOD': ['food', 'agriculture', 'nutrition'],
    'HEALTH': ['health', 'mortality', 'life expectancy']
}

# function to categorize indicators
def categorize_indicator(indicator_name):
    indicator_lower = indicator_name.lower()
    for category, keywords in indicator_categories.items():
        if any(keyword in indicator_lower for keyword in keywords):
            return category
    return 'OTHER'

# add category column
combined_data['Category'] = combined_data['Indicator Name'].apply(categorize_indicator)

# count indicators per category
category_counts = combined_data['Category'].value_counts()
print("Indicators per category:")
print(category_counts)

In [ ]:
import matplotlib.pyplot as plt

# pivot the data to create a matrix with countries as rows and indicators as columns
pivot_df = combined_data.pivot_table(
    index=['Country Name', 'Year', 'PM10'],
    columns='Indicator Name',
    values='WB Value'
).reset_index()

# select columns with at least 50% non-null values
threshold = len(pivot_df) * 0.5
pivot_df = pivot_df.loc[:, pivot_df.notna().sum() > threshold]

# standardize the data (excluding PM10)
columns_to_standardize = [col for col in pivot_df.columns if col not in ['Country Name', 'Year', 'PM10']]
scaler = StandardScaler()
pivot_df[columns_to_standardize] = scaler.fit_transform(pivot_df[columns_to_standardize].fillna(0))

# calculate correlation with PM10
correlations = pivot_df[columns_to_standardize].corrwith(pivot_df['PM10']).sort_values(ascending=False)

# plot top correlations
plt.figure(figsize=(12, 6))
correlations.head(10).plot(kind='bar')
plt.title('Top 10 Indicators Correlated with PM10')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("\nTop 5 positively correlated indicators with PM10:")
print(correlations.head().to_frame('correlation'))

print("\nTop 5 negatively correlated indicators with PM10:")
print(correlations.tail().to_frame('correlation'))

In [ ]:
from sklearn.impute import SimpleImputer

# select numeric columns
numeric_columns = pivot_df.select_dtypes(include=[np.number]).columns
# create mean imputer
imputer = SimpleImputer(strategy='mean')
# impute missing values
imputed_data = imputer.fit_transform(pivot_df[numeric_columns])
imputed_df = pd.DataFrame(imputed_data, columns=numeric_columns)

# standardize the imputed numeric data
scaler = StandardScaler()
standardized_data = scaler.fit_transform(imputed_df)
standardized_df = pd.DataFrame(standardized_data, columns=numeric_columns)

# add back non-numeric columns
final_df = pd.concat([pivot_df[['Country Name', 'Year']], standardized_df], axis=1)

print("Data shape after cleaning:", final_df.shape)
print("\nFirst few rows of cleaned and standardized data:")
final_df.head()

In [ ]:
from sklearn.decomposition import PCA

# select trade-related indicators
trade_indicators = [col for col in numeric_columns if 'export' in col.lower()]

# perform PCA
pca = PCA()
trade_data = standardized_df[trade_indicators]
trade_pca = pca.fit_transform(trade_data)

# explained variance
print("Explained variance ratio:")
print(pca.explained_variance_ratio_)
# this is a scree plot but what IS a scree plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), 
         np.cumsum(pca.explained_variance_ratio_), 'bo-')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('PCA Scree Plot')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# economic indicators
economic_indicators = [col for col in standardized_df.columns 
                      if any(term in col.lower() 
                            for term in ['gdp', 'trade', 'export', 'import'])]

# prepare data for clustering
X = standardized_df[economic_indicators]

# perform K-means clustering
n_clusters = 3  # Example number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(X)
# calculate silhouette score
silhouette_avg = silhouette_score(X, clusters)
# cluster labels
final_df['Cluster'] = clusters

# correlation matrix for economic indicators
correlation_matrix = X.corr()

# correlation heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix of Economic Indicators')
plt.tight_layout()
plt.show()

print("\nSilhouette Score:", silhouette_avg)
print("\nCluster Sizes:")
print(pd.Series(clusters).value_counts())

In [ ]:
# calculate cluster characteristics
cluster_means = final_df.groupby('Cluster')[economic_indicators].mean()
# standardize the values for visualization
cluster_means_std = (cluster_means - cluster_means.mean()) / cluster_means.std()

# heatmap of cluster characteristics
plt.figure(figsize=(15, 8))
sns.heatmap(cluster_means_std, cmap='RdYlBu', center=0, annot=True, fmt='.2f')
plt.title('Standardized Characteristics of Each Cluster')
plt.ylabel('Cluster')
plt.tight_layout()
plt.show()

# print cluster membership
print("\nCluster Memberships:")
print(final_df[['Country Name', 'Year', 'Cluster']].sort_values('Cluster'))

# calculate top distinguishing features for each cluster
print("\nTop Distinguishing Features per Cluster:")
for cluster in range(3):
    cluster_data = cluster_means.loc[cluster]
    top_features = cluster_data.sort_values(ascending=False).head(3)
    print(f"\nCluster {cluster} top features:")
    print(top_features)
    # this is not showing the top associated indicators for cluster 2 ? why